# Personalization Agent Demo

## Connect to the Weaviate Cloud instance

> Reminder: Weaviate Agents are only available for Weaviate Cloud instances.

Connect to your Weaviate instance, using credentials from the Weaviate Cloud console. Here, they are loaded from the `.env` file.

In [ ]:
import weaviate
from weaviate.classes.init import Auth
import os
from dotenv import load_dotenv

weaviate_url = os.getenv("WEAVIATE_URL")
weaviate_key = os.getenv("WEAVIATE_API_KEY")
load_dotenv()

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url, auth_credentials=Auth.api_key(weaviate_key)
)

## Connect to the Personalization Agent

You can initialize the Personalization Agent, or connect to an existing one, as shown below.

In [ ]:
from weaviate.agents.personalization import PersonalizationAgent
from weaviate.classes.config import DataType
from helpers import PA_DEMO_COLLECTION

if PersonalizationAgent.exists(client, PA_DEMO_COLLECTION):
    pa = PersonalizationAgent.connect(client=client, reference_collection=PA_DEMO_COLLECTION)
else:
    pa = PersonalizationAgent.create(
        client=client,
        reference_collection=PA_DEMO_COLLECTION,
        # These properties will be used to give the persona a "baseline" set of characteristics
        user_properties={
            "age": DataType.NUMBER,
            "favorite_genres": DataType.TEXT_ARRAY,
            "favorite_years": DataType.NUMBER_ARRAY,
            "language": DataType.TEXT,
        },
    )

## Create a persona

In [ ]:
from weaviate.agents.classes import Persona
from weaviate.util import generate_uuid5
from uuid import uuid4  # If you want to generate a random UUID

persona_id = generate_uuid5("sebawita")  # To generate a deterministic UUID
# persona_id = uuid4()  # To generate a random UUID

# You can delete a persona if you want to remove it from the system
pa.delete_persona(persona_id)

if pa.has_persona(persona_id):
    print(f"Persona with ID {persona_id} already exists.")
else:
    print(f"Creating new persona with ID {persona_id}.")
    pa.add_persona(
        Persona(
            persona_id=persona_id,
            properties={
                "age": 30,
                "favorite_genres": ["Sci-Fi", "Horror", "Mystery"],
                "favorite_years": [1999, 1996, 2008, 2019],
                "language": "English",
            },
        )
    )

In [ ]:
pa.get_persona(persona_id)

## Add interactions

This is how the agent learns each persona's preferences. (At least one interaction is required to perform queries.)

In [ ]:
from weaviate.agents.classes import PersonaInteraction
from weaviate.collections.classes.filters import Filter
from helpers import get_movie_uuid  # Helper to get the UUID of a movie

# Need at least one interaction to get recommendations
pa.add_interactions(interactions=[
    PersonaInteraction(
        persona_id=persona_id, item_id=get_movie_uuid(client, "Avatar"), weight=0.5
    ),
])

## Queries

We can already perform queries.

### Basic queries

Fast, most basic personalized queries

- Uses vectors of interaction history only

In [ ]:
response = pa.get_objects(persona_id, limit=50, use_agent_ranking=False)

In [ ]:
from helpers import print_movie_response_details

print_movie_response_details(response, 5)

### Agent reranking

The agent can smartly rerank the results based on the information about the persona, as well as the interactions.

In [ ]:
response = pa.get_objects(persona_id, limit=50, use_agent_ranking=True)

print_movie_response_details(response, 5)

### With Reranker + Instruction

- The most complex personalized queries
- Uses vectors of interaction history and AI-based reranker
- Instructions used to guide the reranker

In [ ]:
response = pa.get_objects(
    persona_id,
    limit=50,
    use_agent_ranking=True,
    instruction="I'm looking for something for the whole family, maybe historical."
)

print_movie_response_details(response, 5)

### Add more interactions

Over time, you will add more interactions to the agent, which will help it learn more about the persona's preferences.

Note each interaction can be positive or negative. 
(1: most positive, 0: neutral, -1: most negative)

In [ ]:
interactions = [
    PersonaInteraction(
        persona_id=persona_id, item_id=get_movie_uuid(client, "How It Ends"), weight=0.7
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=get_movie_uuid(client, "Doctor Sleep"), weight=0.6
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=get_movie_uuid(client, "The Howling"), weight=0.5
    ),
    PersonaInteraction(
        persona_id=persona_id,
        item_id=get_movie_uuid(client, "A Nightmare on Elm Street"),
        weight=0.9,
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=get_movie_uuid(client, "High Life"), weight=0.75
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=get_movie_uuid(client, "Magic Mike"), weight=-0.9
    ),  # negative
]

pa.add_interactions(interactions=interactions)

### Retry queries

In [ ]:
response = pa.get_objects(persona_id, limit=50, use_agent_ranking=False)

print_movie_response_details(response, 5)

### With Reranker + Instruction + Filter

- The most complex personalized queries
- Uses vectors of interaction history and AI-based reranker
- Instructions used to guide the reranker
- Filters out items that are not relevant to the user

In [ ]:
# With Reranker + Instruction + Filter
response = pa.get_objects(
    persona_id,
    limit=50,
    use_agent_ranking=True,
    instruction="User's favorite sub genre of horror is slasher and least favorite is space horror",
    filters=Filter.by_property("genres").contains_any(["Mystery", "Horror"]),
)

In [ ]:
print_movie_response_details(response, 10)

## Compare results 

What happens if we look at results across different query methods?

In [ ]:
from helpers import compare_genre_match_scores

query = pa.query(persona_id=persona_id, strength=0.95)  # fully personalized reranking
query2 = pa.query(persona_id=persona_id, strength=0.0)  # no reranking

response1 = query.hybrid("slasher", limit=50)
response2 = query2.hybrid("slasher", limit=50)

compare_genre_match_scores(
    [response1, response2],
    preferred_genres={"Science Fiction", "Horror", "Mystery"},
    top_n=10,
    response_labels=["Personalized", "No Personalization"],
)

We see that the personalized results are much more relevant to the user than the non-personalized results.

In [ ]:
client.close()